# Implementação do Algorítmo Genético Plano para o Problemas das Oito Rainhas

## Implementação do Ambiente
Primeiramente, implementei as regras do ambiente na classe EightQueen.

In [2]:
import random
class EightQueen():
    def __init__(self, n=8):
        self.board = [[0 for _ in range(n)] for _ in range(n)]
        for i in range(n):
            self.board[i][random.randint(0, n-1)] = 1
        self.n = n

    def deepcopy(self):
        nt = EightQueen(self.n)
        for i in range(self.n):
            for j in range(self.n):
                nt.board[i][j] = self.board[i][j]
        return nt

    def move(self, c, p):
        for i in range(self.n):
            self.board[c][i] = 0            
        self.board[c][p] = 1
        
    def find_row(self, c):
        row = -1
        for j in range(self.n): #verifica a linha da rainha na coluna i
            if self.board[c][j] == 1:
                row = j
                break
        return row
    
    def evaluate(self):
        score = 0
        for i in range(self.n):
            rowi = self.find_row(i)
            for k in range(i+1, self.n): #verifica colunas que colidem com a ccolun i
                if k != i:
                    rowk = self.find_row(k)
                    if rowk != rowi and abs(rowk - rowi) != abs(i - k):
                        score += 1
        return score
    
    def neighborhood(self):
        #Para cada coluna c
            #movimento a rainha para uma posição ainda não oculpada nesta coluna.
        neigh = []
        for i in range(self.n):
            rowi = self.find_row(i)
            for j in range(self.n):
                if j != rowi:
                    nn = self.deepcopy()
                    nn.move(i, j)
                    neigh.append(nn)
        return neigh

    def render(self):
        for i in range(self.n):
            for j in range(self.n):
                print(self.board[j][i], end=" ")
            print()

In [28]:
class Genome():
    def __init__(self, code):
        self.code = code
        self.fitness = 0
    
    def mutate(self, ratio):
        if random.random() < ratio:
            c = random.randint(0, 7)
            p = random.randint(0, 7)
            self.code[c] = p

class AG:
    def __init__(self, k=10, mutation_ratio=0.01):
        self.mutation_ratio = mutation_ratio
        self.population = []
        # [1, 1, 2, 3, 4, 5, 7, 6]
        for i in range(k):
            g = Genome([random.randint(0, 7) for _ in range(8)])
            self.population.append(g)
    
    def decoding(self, g):
        board = EightQueen()
        for c in range(8):
            board.move(c, g.code[c])
        return board

    def classification(self):
        for g in self.population:
            q = self.decoding(g)
            g.fitness = q.evaluate()
        self.population.sort(key = lambda g: g.fitness)
        self.population.reverse()

    def selection(self):
        fitness_sum = sum([g.fitness for g in self.population])
        p_start = 0
        selected = []
        while  (len(selected) < len(self.population)):
            r = random.random()
            s = 0
            for g in self.population:
                p = g.fitness/fitness_sum                
                s = s + p
                if r < s:
                    #print("P ", p)
                    selected.append(g)
                    break 
        return selected
    
    def crossover(self, selected):
        pop = []
        for i in range(0, len(selected), 2):
            g1 = selected[i]
            g2 = selected[i+1]
            idx = random.randint(0, len(g1.code)-1)
            c1 = [0]*8
            c2 = [0]*8
            c1[0:idx] = g1.code[0:idx]
            c1[idx:] = g2.code[idx:]

            c2[0:idx] = g2.code[0:idx]
            c2[idx:] = g1.code[idx:]
            pop.append(Genome(c1))
            pop.append(Genome(c2))
        self.population = pop
        return pop
    
    def mutation(self):
        for g in self.population:
            g.mutate(self.mutation_ratio)



In [29]:
ag = AG(4)
ag.population = [Genome([1,3,6,3,7,4,4,1]), 
                    Genome([2,1,6,4,1,3,0,0]), 
                    Genome([1,3,3,0,4,0,1,3]),
                    Genome([2,1,4,3,2,1,0,2])]
ag.classification()
selected = ag.selection()
print("=============== FITNESS ========================")
for g in  ag.population:
    print(g.fitness)
print("============== SELECTED GENOMES ========================")
for g in selected:
    print(g.code)
print("============= CROSSOVER RESULTS =================")
pop = ag.crossover(selected)
for g in pop:
    print(g.code)
ag.mutation()
print("============= MUTATION RESULTS ===================")
for g in pop:
    print(g.code)

=============== FITNESS ========================
24
23
20
11
============== SELECTED GENOMES ========================
[2, 1, 6, 4, 1, 3, 0, 0]
[1, 3, 6, 3, 7, 4, 4, 1]
[1, 3, 6, 3, 7, 4, 4, 1]
[1, 3, 6, 3, 7, 4, 4, 1]
============= CROSSOVER RESULTS =================
[2, 3, 6, 3, 7, 4, 4, 1]
[1, 1, 6, 4, 1, 3, 0, 0]
[1, 3, 6, 3, 7, 4, 4, 1]
[1, 3, 6, 3, 7, 4, 4, 1]
============= MUTATION RESULTS ===================
[2, 3, 6, 3, 7, 4, 4, 1]
[1, 1, 6, 4, 1, 3, 0, 0]
[1, 3, 6, 3, 7, 4, 4, 1]
[1, 3, 6, 3, 7, 4, 4, 1]


In [34]:
ag = AG(10, 0.005)
#ag.population = [Genome([1,3,6,3,7,4,4,1]), 
#                    Genome([2,1,6,4,1,3,0,0]), 
#                    Genome([1,3,3,0,4,0,1,3]),
#                    Genome([2,1,4,3,2,1,0,2])]

def mean_fitness(pop):
    s = 0
    for g in pop:
        s += g.fitness
    return s/len(pop)

for i in range(1000):
    ag.classification()
    print(mean_fitness(ag.population))
    selected = ag.selection()
    pop = ag.crossover(selected)
    ag.mutation()


21.3
20.4
19.8
19.9
20.1
21.1
21.5
22.2
21.5
21.7
22.4
22.7
22.8
21.9
21.5
21.1
20.6
19.8
19.8
20.2
20.4
21.0
20.7
21.5
21.2
21.2
21.0
21.7
21.6
21.8
21.8
21.8
21.8
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
21.4
20.8
20.8
21.4
22.0
22.1
22.1
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
21.9
21.7
21.6
21.7
21.9
22.0
22.0
21.7
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
22.0
21.9
21.6
21.4
21.0
21.0
21.0
21.2
20.8
20.9
21.1
21.2
21.7
21.8
21.9
21.8
21.8
21.8
21.8
21.7
21.6
21.7
